In [1]:
import napari
import os
import glob
from pathlib import Path
from tifffile import imread
import numpy as np
import csv
import pandas as pd
from qtpy.QtWidgets import QComboBox, QPushButton, QSlider
%gui qt


In [12]:
class EventViewer(object):
    
    def __init__(self, viewer, imagename, Name, csv_dir, save = False, newimage = True):
     
          self.save = save
          self.newimage = newimage
          self.viewer = viewer
           
          print('reading image')      
          self.imagename = imagename  
          self.image = imread(imagename)
          print('image read')
          self.Name = Name
            
          self.csv_dir = csv_dir
          
          
          self.click()
    def click(self):

        
                Cadherine = 'BestCad'
                Nuclei = 'BestNuclei'
                TotalOutofFocus = 'Totaloff'
                CadherineOutofFocus = 'Cadoff'
                NucleiOutofFocus = 'Nucleioff'
                if self.save == True:
 
                        Caddata = self.viewer.layers[Cadherine].data
                        caddf = pd.DataFrame(Caddata, index = None, columns = ['Z', 'Y', 'X'])
                        caddf.to_csv(self.csv_dir + '/' + 'ONEAT' + Cadherine + self.Name +  '.csv', index = False, mode = 'a')

                        Nucleidata = self.viewer.layers[Nuclei].data
                        nucdf = pd.DataFrame(Nucleidata, index = None, columns = ['Z', 'Y', 'X'])
                        nucdf.to_csv(self.csv_dir + '/' +'ONEAT' + Nuclei + self.Name +  '.csv', index = False, mode = 'a')

                        TotalOutofFocusdata = self.viewer.layers[TotalOutofFocus].data
                        offdf = pd.DataFrame(TotalOutofFocusdata, index = None, columns = ['Z', 'Y', 'X'])
                        offdf.to_csv(self.csv_dir + '/' + 'ONEAT' +TotalOutofFocus + self.Name +  '.csv', index = False, mode = 'a')

                        CadherineOutofFocusdata = self.viewer.layers[CadherineOutofFocus].data
                        cadoffdf = pd.DataFrame(CadherineOutofFocusdata, index = None, columns = ['Z', 'Y', 'X'])
                        cadoffdf.to_csv(self.csv_dir + '/' + 'ONEAT' +CadherineOutofFocus + self.Name +  '.csv', index = False, mode = 'a')

                        NucleiOutofFocusdata = self.viewer.layers[NucleiOutofFocus].data
                        nucoffdf = pd.DataFrame(NucleiOutofFocusdata, index = None, columns = ['Z', 'Y', 'X'])
                        nucoffdf.to_csv(self.csv_dir + '/' + 'ONEAT' +NucleiOutofFocus + self.Name +  '.csv', index = False, mode = 'a')

        
        
                if self.newimage == True:
                     for layer in list(self.viewer.layers):

                            self.viewer.layers.remove(layer) 
                    
                if self.save == False:
                        self.viewer.add_image(self.image, name = self.Name)


                        # add the first points layer for BestZCadherine Z point
                        self.viewer.add_points(name= Cadherine, face_color='red', ndim = 3)

                        # add the second points layer for BestZNuclei Z point
                        self.viewer.add_points(name= Nuclei, face_color='blue', ndim = 3)

                        # add the second points layer for TotalOutofFocus Z point
                        self.viewer.add_points(name= TotalOutofFocus, face_color='green', ndim = 3)

                        # add the second points layer for CadherineOutofFocus Z point
                        self.viewer.add_points(name= CadherineOutofFocus, face_color='yellow', ndim = 3)

                        # add the second points layer for TotalOutofFocus Z point
                        self.viewer.add_points(name= NucleiOutofFocus, face_color='white', ndim = 3)

                        # programatically enter add mode for both Points layers to enable editing

                        self.viewer.layers[Cadherine].mode = 'add'
                        self.viewer.layers[Nuclei].mode = 'add'
                        self.viewer.layers[TotalOutofFocus].mode = 'add'
                        self.viewer.layers[CadherineOutofFocus].mode = 'add'
                        self.viewer.layers[NucleiOutofFocus].mode = 'add'

                return self.imagename
                

# In the cell below we create the training data for focus oneat. There are 5 categories:

1) TotalOutofFocus: Here neither nuclei nor cadherine is in focus
2) BestZNuclei: This is the best Z plane for Nuclei to be in focus
3) BestZCadherine: This is the best Z plane for Cadherine to be in focus
4) CadherineOurofFocus: In these planes Cadherine is out of focus
5) NucleiOutofFocus: In these plances Nuclei is out of focus

To add points for each class, select the desired points layer from the Napari window and start adding points.

To remove points press "s" and select the point to remove of make a rectangle to remove bunch of points at once, then hit backspace to remove those points. To get back to add mode press "p" and start adding points. To select all poitns in the current Z frame hit a. Once you have made the selection of all the desired classes close the Napari window and it will save the csv file of the clicks and open the next image present in the folder.

In [13]:
sourcedir = '/home/sancere/Kepler/FinalONEATTraining/NEATTest/SmallTest/'
csv_dir = '/home/sancere/Kepler/FinalONEATTraining/NEATTest/'
Imageids = []
Boxname = 'ImageIDBox'
Path(csv_dir).mkdir(exist_ok = True)



Raw_path = os.path.join(sourcedir, '*tif')
X = glob.glob(Raw_path)
for imagename in X:
         Imageids.append(imagename)
         Name = os.path.basename(os.path.splitext(imagename)[0])

imageidbox = QComboBox()   
imageidbox.addItem(Boxname)   
tracksavebutton = QPushButton('Save Clicks')

for i in range(0, len(Imageids)):


     imageidbox.addItem(str(Imageids[i]))
        
        
viewer = napari.Viewer()        
viewer.window.add_dock_widget(imageidbox, name="Image", area='bottom')    
viewer.window.add_dock_widget(tracksavebutton, name="Save Clicks", area='bottom')
imageidbox.currentIndexChanged.connect(
         lambda trackid = imageidbox: EventViewer(
                 viewer,
                  imageidbox.currentText(),
                       os.path.basename(os.path.splitext(imageidbox.currentText())[0]), csv_dir, False, True ))     

tracksavebutton.clicked.connect(
        lambda trackid= tracksavebutton:EventViewer(
                 viewer,
                  imageidbox.currentText(),
                       os.path.basename(os.path.splitext(imageidbox.currentText())[0]), csv_dir, True, False )) 



print(test)

reading image
image read


/home/sancere/anaconda3/envs/tensorflowGPU/lib/python3.7/site-packages/napari/_vispy/vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


reading image
image read
